Практика 8: Методы защиты от атак на модели ИИ

Шаг 1: Защита с помощью Adversarial Training

In [32]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.datasets import mnist

# Загрузка данных MNIST
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# Нормализация данных
train_images = train_images / 255.0
test_images = test_images / 255.0

# Преобразование меток в one-hot encoding
train_labels = tf.keras.utils.to_categorical(train_labels, 10)
test_labels = tf.keras.utils.to_categorical(test_labels, 10)

# Уменьшаем количество обучающих изображений
train_images = train_images[:1000]
train_labels = train_labels[:1000]

# Функция FGSM атаки
def fgsm_attack(image, epsilon, gradient):
    perturbation = epsilon * np.sign(gradient)
    adversarial_image = image + perturbation
    adversarial_image = np.clip(adversarial_image, 0, 1)
    return adversarial_image

# Функция для генерации противоречивых примеров Мелкумян
def generate_adversarial_examples(model, images, labels, epsilon):
    adversarial_images = []
    for i in range(len(images)):
        image = tf.convert_to_tensor(images[i].reshape((1, 28, 28, 1)), dtype=tf.float32)
        label = tf.convert_to_tensor(labels[i].reshape((1, 10)), dtype=tf.float32)
        with tf.GradientTape() as tape:
            tape.watch(image)
            prediction = model(image)
            loss = tf.keras.losses.categorical_crossentropy(label, prediction)
        gradient = tape.gradient(loss, image)
        adversarial_image = fgsm_attack(image.numpy(), epsilon, gradient.numpy())
        adversarial_images.append(adversarial_image.reshape(28, 28))
    return np.array(adversarial_images)

# Создание модели
def create_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Обучение модели с противоречивыми примерами
def adversarial_training(model, train_images, train_labels, epsilon):
    for epoch in range(5):  # Короткое обучение для демонстрации
        for i in range(0, len(train_images), 32):
            batch_images = train_images[i:i+32]
            batch_labels = train_labels[i:i+32]
            adversarial_images = generate_adversarial_examples(model, batch_images, batch_labels, epsilon)
            combined_images = np.concatenate([batch_images, adversarial_images], axis=0)
            combined_labels = np.concatenate([batch_labels, batch_labels], axis=0)
            model.train_on_batch(combined_images, combined_labels)

# Инициализация модели
model = create_model()

# Тренировка модели с защитой на противоречивых примерах
adversarial_training(model, train_images, train_labels, epsilon=0.1)

# Сохранение защищенной модели
model.save('adversarial_trained_model.h5')

Шаг 2: Градиентная маскировка (Gradient Masking)

In [33]:
from tensorflow.keras.layers import Activation

# Обновление модели для градиентной маскировки
def create_masked_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(10),
        Activation('softplus')  # Используем softplus вместо softmax для градиентной маскировки
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Обучение модели с градиентной маскировкой
masked_model = create_masked_model()
masked_model.fit(train_images, train_labels, epochs=5)

# Сохранение модели с градиентной маскировкой
masked_model.save('masked_model.h5')

Epoch 1/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.4317 - loss: 1.9380
Epoch 2/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8153 - loss: 0.7766
Epoch 3/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8796 - loss: 0.4449
Epoch 4/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9178 - loss: 0.2964
Epoch 5/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9489 - loss: 0.2506


Шаг 3: Регуляризация и нормализация для повышения устойчивости
Использование таких методов, как L2-регуляризация, дропаут и нормализация батчей, может помочь
улучшить устойчивость модели к атакам.


In [34]:
# Модель с регуляризацией и нормализацией
def create_regularized_model():
 model = tf.keras.Sequential([
 tf.keras.layers.Flatten(input_shape=(28, 28)),
 tf.keras.layers.Dense(128, activation='relu',
kernel_regularizer=tf.keras.regularizers.l2(0.01)),
 tf.keras.layers.Dropout(0.5),
 tf.keras.layers.BatchNormalization(),
 tf.keras.layers.Dense(10, activation='softmax')
 ])
 model.compile(optimizer='adam', loss='categorical_crossentropy',
metrics=['accuracy'])
 return model
# Обучение модели с регуляризацией и нормализацией
regularized_model = create_regularized_model()
regularized_model.fit(train_images, train_labels, epochs=5)
# Сохранение модели с регуляризацией
regularized_model.save('regularized_model.h5')


Epoch 1/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.2874 - loss: 4.1493
Epoch 2/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7541 - loss: 2.1019
Epoch 3/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8095 - loss: 1.6798
Epoch 4/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8869 - loss: 1.3960
Epoch 5/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8823 - loss: 1.2848


Шаг 4: Оценка моделей на противоречивых примерах
Теперь проверим эффективность всех защитных методов на атакованных данных, созданных с
помощью FGSM и других методов, таких как PGD или GAN.


In [37]:
# Оценка защищенных моделей на противоречивых примерах
adversarial_test_images = generate_adversarial_examples(model, test_images, test_labels, epsilon=0.1)

print("Base Model Accuracy on Adversarial Examples:") #Базовая модель
model.evaluate(adversarial_test_images, test_labels)

print('Мелкумян')

print("Adversarially Trained Model Accuracy:") #Модель с использованием противоречивых примеров
adv_model = tf.keras.models.load_model('adversarial_trained_model.h5')
adv_model.evaluate(adversarial_test_images, test_labels)

print("Masked Model Accuracy:") #Модель с градиентной маскировкой
masked_model = tf.keras.models.load_model('masked_model.h5')
masked_model.evaluate(adversarial_test_images, test_labels)

print("Regularized Model Accuracy:")#Модель с регуляризацией и нормализацией
regularized_model = tf.keras.models.load_model('regularized_model.h5')
regularized_model.evaluate(adversarial_test_images, test_labels)



Base Model Accuracy on Adversarial Examples:
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.4913 - loss: 1.4732


Мелкумян
Adversarially Trained Model Accuracy:
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.4913 - loss: 1.4732


Masked Model Accuracy:
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.4614 - loss: 1.6262
Regularized Model Accuracy:


313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.5221 - loss: 2.2077


[2.0972368717193604, 0.5655999779701233]

Базовая модель показала точность на уровне 49% при оценке на противоречивых примерах. Это демонстрирует ее уязвимость к подобным атакам.

Модель с использованием тренировок на противоречивых примерах (Adversarial Training) также достигла точности 49%, что указывает на отсутствие значительного улучшения при использовании этого метода в текущей реализации.

Модель с градиентной маскировкой показала снижение точности до 46%. Это говорит о том, что данный метод защиты оказался менее эффективным и, возможно, требует доработки или не подходит для данного типа атак.

Модель с регуляризацией и нормализацией продемонстрировала наилучший результат среди всех протестированных методов, достигнув точности 55% на противоречивых примерах. Это на 6% выше базового результата и указывает на эффективность данного подхода.